In [ ]:
%load_ext autoreload
%autoreload 2

from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import seaborn as sns
from models.transformer import Transformer
from train import Trainer
from dataloader import EnglishToSpanish

# Model

In [ ]:
trainer = Trainer()
data = EnglishToSpanish()

In [ ]:
tm = Transformer()

In [ ]:
trainer.fit(tm, data)

In [ ]:
dl = iter(DataLoader(data))

In [179]:
inp, out, src, tgt = next(dl)
sos = torch.repeat_interleave(data.sos, 1, 0)
out1 = torch.cat([sos], 1)
print(out1)

tensor([[0]])


In [180]:
data._idx_to_token(out)

[['Por',
  'el',
  'presidente',
  'Thomas',
  'S.',
  'Monson',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>']]

In [181]:
data._idx_to_token(inp, is_en=True)

[['By',
  'President',
  'Thomas',
  'S.',
  'Monson',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>']]

In [184]:
R = tm(inp, out1)
last_pred = R[0,-1]
print(f"Adding with prob. {torch.max(last_pred)}")
r = torch.argmax(last_pred)
out1 = torch.cat([out1, r.reshape(1, 1)], 1)
print(out1)

Adding with prob. 0.999320387840271
tensor([[    0, 11413,     2,     2]])


In [185]:
data._idx_to_token(out1)

[['<SOS>', '¿', '<PAD>', '<PAD>']]